In [1]:
import tensorflow as tf
from data_generator.batch_generator_functions import unstack_batches

In [2]:
from session.cli_parser import CLIParser
from session.session import Session

cli_args = ['-id', '123406789013', 
        '-l', 'tf_cup_f1_score_loop_based', '-lp', 'beta', '0.4', 
        '-lr', '0.001', '-e', '2', '-o', 'Adam', 
        '-mc', 'UeberNet', '-mp', 'best2',
        '-tg', 'tune_train', '-vg', 'tune_val',
        '-svtb', 'False', '-svcp', 'False',
        '-svhis', 'False', '-svm', 'False'
       ]
    
parser = CLIParser()
args = parser.parse_args(cli_args)

session = Session(args, auto_init=False)
session.initialize()

2022-06-26 18:29:38.998308   [Session] Initializing ...
2022-06-26 18:29:38.999346   [Session] Syncing existing training runs ...
2022-06-26 18:29:39.000339   [Session] Checking for previously run epoch logs and stored weights ...
2022-06-26 18:29:39.015615   [Session] Found log indicating no previous training (epoch: 0).
2022-06-26 18:29:39.016624   [Session] Synced logs and weights at epoch 0.
2022-06-26 18:29:39.016624   [Session] Finished initialization.
2022-06-26 18:29:39.017673   [Session] Started initialize subroutine ...
2022-06-26 18:29:39.017673   [Session] Retrieving model ...
2022-06-26 18:29:43.503775   [Session] Retrieved model with 3448285 parameters successfully.
2022-06-26 18:29:43.503775   [Session] Compiling model ...
2022-06-26 18:29:43.520781   [Session] Compiled model successfully.
2022-06-26 18:29:43.520781   [Session] Initializing data generators ...
2022-06-26 18:29:43.521900   [UtilFunctions] Found an existing folder for C:\Users\ricof\Documents\AI-Cup 2022\t

In [3]:
# https://stackoverflow.com/questions/65058699/using-gradients-in-a-custom-loss-function-tensorflowkeras
epochs = session.epochs

X_unstacked = unstack_batches(session.X_train, session.batch_size)[:5, :8]
y_unstacked = unstack_batches(session.y_train, session.batch_size)[:5, :8]

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (X_batch, y_batch) in enumerate(zip(X_unstacked, y_unstacked)):
    
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape(persistent=True) as tape:
            X_tensor = tf.convert_to_tensor(X_batch, dtype=tf.float64)
            tape.watch(X_tensor)
            
            # feed forwaard
            y_pred = session.model(X_tensor, training=True)
            
            # Gradient and the corresponding loss function
            loss_value = session.model.loss(y_pred, y_batch)
            print(loss_value)
    
        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        # gradients = tape.gradient(loss_value, session.model.trainable_weights)
        gradients = tape.gradient(y_pred, X_tensor)
        
        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        session.model.optimizer.apply_gradients(zip(gradients, session.model.trainable_weights))
    
        # Log every 20 batches.
        if step % 1 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * session.batch_size))


Start of epoch 0
tf.Tensor(0.0008561644, shape=(), dtype=float32)


InvalidArgumentError: cannot compute ResourceApplyAdam as input #9(zero-based) was expected to be a float tensor but is a double tensor [Op:ResourceApplyAdam]

In [17]:
import tensorflow.keras.backend as K

def f1_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1 - y_true)*(1 - y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [18]:
from session.cli_parser import CLIParser
from session.session import Session

cli_args = ['-id', '123406789013', 
        '-l', 'tf_cup_f1_score_loop_based', '-lp', 'beta', '0.4', 
        '-lr', '0.001', '-e', '2', '-o', 'Adam', 
        '-mc', 'UeberNet', '-mp', 'best2',
        '-tg', 'tune_train', '-vg', 'tune_val',
        '-svtb', 'False', '-svcp', 'False',
        '-svhis', 'False', '-svm', 'False'
       ]
    
parser = CLIParser()
args = parser.parse_args(cli_args)

session = Session(args, auto_init=False)

2022-06-26 21:23:08.027508   [Session] Initializing ...
2022-06-26 21:23:08.027508   [Session] Syncing existing training runs ...
2022-06-26 21:23:08.027508   [Session] Checking for previously run epoch logs and stored weights ...
2022-06-26 21:23:08.035205   [Session] Found log indicating no previous training (epoch: 0).
2022-06-26 21:23:08.035205   [Session] Created temporary checkpoint directory C:\Users\ricof\Documents\AI-Cup 2022\models\123406789013\ckpts\ ...
2022-06-26 21:23:08.035205   [Session] Synced logs and weights at epoch 0.
2022-06-26 21:23:08.035205   [Session] Finished initialization.


In [19]:
session.loss = f1_loss

In [21]:
session.initialize()

In [22]:
session.execute()

2022-06-26 21:23:38.656076   [Session] Starting run in current session with 2 retries left and 2 epochs remaining ...
2022-06-26 21:23:38.656076   [Session] Started training of the model from epoch 0 for 2 epochs ...
Epoch 1/2
106/106 [==============================] - ETA: 0s - loss: 0.4143 - binary_crossentropy: 18891.8672 - dice_loss: 0.3795 - accuracy: 0.5400 - auc: 0.5409 - precision: 0.5405 - recall: 0.9944 - precision_at_recall: 0.5409 - mean_absolute_error: 0.4597
Epoch 1: saving model to C:\Users\ricof\Documents\AI-Cup 2022\models\123406789013\ckpts\cp-0001.ckpt
106/106 [==============================] - 103s 716ms/step - loss: 0.4143 - binary_crossentropy: 18891.8672 - dice_loss: 0.3795 - accuracy: 0.5400 - auc: 0.5409 - precision: 0.5405 - recall: 0.9944 - precision_at_recall: 0.5409 - mean_absolute_error: 0.4597 - val_loss: 0.5160 - val_binary_crossentropy: 12341.0625 - val_dice_loss: 0.4657 - val_accuracy: 0.4250 - val_auc: 0.4250 - val_precision: 0.4250 - val_recall: 1.00